In [37]:
# Author: Marc Padrós Jiménez
# Bachelor's Final Degree Project
# Date: 27th April 2022

# fastai version on laptop is 2.6.0
# On Desktop is 2.5.3

from fastai.vision.all import *
import cv2
import numpy as np

from Libraries.grid_bboxes_library import * 

from Libraries.lungs_detection_library import * 

import keras
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

Get the detected predicted lungs

In [38]:
lungsDetectorModel = setupLungsLearner()

In [39]:
predictedLungs = lungsDetectorModel.get_preds()
# predictedLungs = tuple with 2 elements
# 1st element of the tuple is a TensorBase
# 2nd element of the tuple is a TensorBBox of size nValidationImages x 8 (8 = total number of coords. for image)
# the first 4 coords of a TensorBBox element refer to the bbox of the left lung
# the last 4 coords of a TensorBBox element refer to the bbox of the right lung 

predictedLungs = ((predictedLungs[1] + 1) / 2).numpy()
# (preds + 1) is used to make negative predictions turn into positive 
# NOTE: Why is ((preds + 1) divided by 2? are the predictions too big (I GET THAT IT'S A NORMALIZATION OF PREDICTIONS)
# numpy() converts to a ndarray Example size: (83,4)

Get the grids related to pleural effusion

In [40]:
predictedLungsPaths = lungsDetectorModel.dls.valid_ds.items

gridsPath = "gridsPleuralEffusion/"

# Before storing the grids, remove the content of the path where grids will be stored
import shutil
shutil.rmtree(gridsPath)

for i in range(len(predictedLungsPaths)):
    imageFullName = predictedLungsPaths[i].name # it contains also the extension
    
    imagePath = '../../../Lungs_detection/Datasets/roi_detection_subset/train_val/' + imageFullName

    imgRes = cv2.imread(imagePath) 

    # NOTE: I read again the BW image that was previously readen because 
    # I need it as an array, not as a PILImage type to be able to access the pixels values 
    imgOri = cv2.imread(imagePath, cv2.IMREAD_GRAYSCALE) 

    leftLung = predictedLungs[i][0:4] * [imgOri.shape[0], imgOri.shape[1], imgOri.shape[0], imgOri.shape[1]]
    # convert a TensorBBox to a numpy array, so that, we can access each coord as a flot
    rightLung = predictedLungs[i][4:8] * [imgOri.shape[0], imgOri.shape[1], imgOri.shape[0], imgOri.shape[1]]

    gridsLungLeft = gridBbox(leftLung, imgRes, imgOri, 3, 3) # grid bbox left in the X-ray image / right in the body
    gridsLungRight = gridBbox(rightLung, imgRes, imgOri, 3, 3) # grid bbox right

    #plt.imshow(imgRes)

    #plt.figure()

    #f, axarr = plt.subplots(1,2) 

    imageName = predictedLungsPaths[i].stem # it doesn't contain the extension 

    gridsImagePath = gridsPath + imageName + '/' 

    os.makedirs(gridsImagePath, exist_ok=True)

    #axarr[0].imshow(gridsLungLeft[6])
    cv2.imwrite(os.path.join(gridsImagePath,'D3.png'), gridsLungLeft[6])

    #axarr[1].imshow(gridsLungRight[8])
    cv2.imwrite(os.path.join(gridsImagePath,'E3.png'), gridsLungRight[8])

Classify grids as normal or pleural effusion

In [41]:
def reshape(gridImage):
    im_width = 40
    im_height = 40
    img = gridImage
    img.thumbnail((im_width, im_height)) # scale the image to a size not larger than im_width x im_height
    x = img_to_array(img)  
    x = np.resize(x,(1,40, 40, 1)) # I have to call resize() instead of reshape() because img.thumbnail doesn't return a 40x40 image
    return x

In [42]:
# Load the learned model from the Github repo: https://github.com/ashok133/Pleural-Effusion-Detection
import csv

model = keras.models.load_model("../CNN_model/dataset/train_test/models/effusion_model_v1_scratch.h5")

test_effusion_images_dir = gridsPath

test_effusion_images = [imageFolder for imageFolder in os.listdir(test_effusion_images_dir)]

prediction_effusion = []

# Create a CSV file where to store the pleural effusion predictions results 
with open('pleural_effusion_results_v1_scratch.csv', 'w', newline='') as csvFile:
  fieldnames = ['image_id', 'D3_prediction', 'E3_prediction']
  writer = csv.DictWriter(csvFile, fieldnames=fieldnames)
  writer.writeheader()

  for imageId in test_effusion_images:
    imagePath = test_effusion_images_dir + imageId + '/'
    gridsImages = [load_img(imagePath+gridName, grayscale=True) for gridName in os.listdir(imagePath)] # collect images of the grids where pleural effusion can be located  
    # imageGrids contains one grid image related to the left lung ("D3.png") and another related to the right lung ("E3.png")
    effusion_array = [reshape(grid) for grid in gridsImages] # reshape each grid size to the input size of the CNN "model"
    prediction_effusion.extend("Effusion" if model.predict(effusion)[0][0] > 0.5 else "Normal" for effusion in effusion_array)
    # Update CSV file
    writer.writerow({'image_id': imageId, 'D3_prediction': prediction_effusion[-2], 'E3_prediction': prediction_effusion[-1]})
    # -2 for D3 because D3 is alphabetically before E3, so, it's processed first 